In [ ]:
from tweepy import Stream, OAuthHandler
from tweepy.streaming import StreamListener
from progressbar import ProgressBar, Percentage, Bar
import json
import sys



In [ ]:
#Twitter app information
consumer_secret='YBzLzvRps6bXuTqYwzkS6IvuWxBOFYI6U95dOco8mJrH45KbG9'
consumer_key='uqeurF0j7qCOEKy3Fz0zTLnLH'
access_token='769009179537399808-5m2rIqJZhB1mVRwMkE9AinFFfPOtmNT'
access_token_secret='ojL4HtPBAnFzJr98b5Fla37lBcZsPi3tU5hc1y7aAxXUK'



In [ ]:
#The number of tweets we want to get
max_tweets=100000

#Create the listener class that receives and saves tweets
class listener(StreamListener):
    #On init, set the counter to zero and create a progress bar
    def __init__(self, api=None):
        self.num_tweets = 0
        self.pbar = ProgressBar(widgets=[Percentage(), Bar()], maxval=max_tweets).start()

    #When data is received, do this
    def on_data(self, data):
        #Append the tweet to the 'tweets.txt' file
        with open('/Users/zoec/Documents/cus680/project/twitter2.json', 'a') as tweet_file:
            tweet_file.write(data)
            #Increment the number of tweets
            self.num_tweets += 1
            #Check to see if we have hit max_tweets and exit if so
            if self.num_tweets >= max_tweets:
                self.pbar.finish()
                sys.exit(0)
            else:
                #increment the progress bar
                self.pbar.update(self.num_tweets)
        return True

    #Handle any errors that may occur
    def on_error(self, status):
        print (status)

#Get the OAuth token
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
#Use the listener class for stream processing
twitterStream = Stream(auth, listener())
#Filter for these topics
twitterStream.filter(track=["java","python","sql"])

In [ ]:
import json
import pandas as pd 
tweet_files = ['/Users/zoec/Documents/cus680/project/twitter2.json']
tweets = []
for file in tweet_files:
    with open(file, 'r') as f:
        for line in f.readlines():
            tweets.append(json.loads(line))

In [ ]:
print(tweets)

In [ ]:
df = pd.DataFrame()
df['id'] = list(map(lambda tweet: tweet['id'], tweets))
df['text'] = list(map(lambda tweet: tweet['text'].strip(';').encode('utf-8'), tweets))
df['user'] = list(map(lambda tweet: tweet['user']['screen_name'], tweets))
df['friends_count'] = list(map(lambda tweet: tweet['user']['friends_count'], tweets))
df['statuses_count'] = list(map(lambda tweet: tweet['user']['statuses_count'], tweets))
df['listed_count'] = list(map(lambda tweet: tweet['user']['listed_count'], tweets))
df['followers_count'] = list(map(lambda tweet: tweet['user']['followers_count'], tweets))
df['en']=list(map(lambda tweet: tweet['user']['lang'], tweets))
df

In [ ]:
df['text'] = df['text'].replace(' #', ' ')
df['text'] = df['text'].replace('\n', ' ')
df['text'] = df['text'].replace('\/', ' ')

In [ ]:
df.head(20)

In [ ]:
df.to_csv(r'/Users/zoec/Documents/cus680/project/final_twitter.csv',header= False,index =False,sep = '\t')